In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path = "~/Documents/G3/MA-prediction"
path = expanduser(path)
sys.path.append(path)

import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from MA_prediction.preprocessing import *
from MA_prediction.mkt_calendar import *
from MA_prediction.utils import *

# Data Cleaning 0: Basic
In this notebook we will do very basic data cleaning:

- Change column names from full names to acronyms.
- Transform datatypes: float-like and date-like (to `datetime.date`). 
- Correct some dates 
    - some dates must be trading days. 
    - `dao` to be between $[\text{da}-126, \text{da}]$.
    - create `dr` date of resolution, that is `de` for completed deals and `dw` for withdrawn deals.
- Some manual correction of the dataset, mainly about the deal consideration string.

    
## I/O    
- Input: 
    - `/data/raw/df.csv`.
- Output: 
    - `/data/intermediate/df_basic_cleaning.h5`.
    - `/data/intermediate/df_basic_cleaning.csv`, easier for inspection than the raw dataset.

# Load data and change column names
Full column names in the raw data are too long and unwieldy to carry out python operations; thus we replace them with the acronyms in the database from the report file. Their correspondence is saved as a `csv` file called `column_names.csv`.  Another comprehensive file `SDC_MA_guide.pdf` explains the exact definition of all the variables in the database.

In [3]:
# load raw dataset
filepath = f"{path}/data/raw/df.csv"
df = pd.read_csv(filepath, index_col=0, na_values=['nm', 'np'], low_memory=False)

# extract full column names
colnames_full = list(map(lambda x: " ".join(x.split()).strip(), [df.index.name] + list(df.columns)))

# extract acronyms of variables from the report file; the first name is that of index.
filepath = f"{path}/data/reference/report.rpt"
colnames = extract_colnames_from_report_file(filepath)
# show the last 10 column acronyms
colnames[-10:]

['pricebook',
 'eqvalcf',
 'eqvalsales',
 'eqval',
 'tlia',
 'cass',
 'clia',
 'lockup',
 'dae',
 'vest']

In [4]:
# change column names
df.index.name = colnames[0]
df.columns = colnames[1:]

print_shape(df)
df.tail()

The dataset is of size (12082, 94).


,statc,one_day,aone_day,dao,da,dateannorig_days,de,dateeffexp,dw,definitive_agt,...,pricebook,eqvalcf,eqvalsales,eqval,tlia,cass,clia,lockup,dae,vest
master_deal_no,,,,,,,,,,,,,,,,,,,,,
3992461020,P,10/24/22,12/16/22,10/25/22,12/18/22,54,NaN,12/31/23,NaN,Yes,...,8.659,16.011,2.087,4547.200,"1,748.5","1,001.7",802.9,No,No,No
4015877020,P,12/16/22,NaN,12/19/22,12/19/22,0,NaN,02/28/23,NaN,Yes,...,4.839,NaN,0.752,16.141,18.3,14.2,16.4,No,No,No
4016515020,P,12/14/22,12/19/22,12/20/22,12/20/22,0,NaN,06/30/23,NaN,Yes,...,NaN,NaN,NaN,52.581,NaN,NaN,NaN,No,No,No
4017224020,P,12/20/22,12/20/22,12/21/22,12/21/22,0,NaN,03/31/23,NaN,Yes,...,0.750,NaN,2.912,55.152,61.3,97.6,11.2,No,No,No
4019588020,P,12/23/22,NaN,12/27/22,12/27/22,0,NaN,NaN,NaN,No,...,NaN,NaN,0.895,25.412,52.4,34.1,36.4,No,No,No


In [5]:
# save the correspondence between acronym and full name for convenience
filepath = f"{path}/data/reference/column_names.csv"
pd.Series(colnames_full, index=colnames, name='column name').to_csv(filepath)

# Transform float-like and date-like datatypes

In [6]:
# numeric-like columns to transform
cols_float = ['val', 'mv', 'amv', 'pr', 'ppmday', 'ppmwk', 'ppm4wk', 'roe', 'tlia', 'cass', 'clia']
# apply function to each column
df[cols_float] = df[cols_float].apply(convert_num_str_ser_to_float)

In [7]:
# date-like columns to transform
cols_dt = ['one_day', 'aone_day', 'dao', 'da', 'de', 'dateeffexp', 'dw', 'da_date', 'dateval', 'dcom', 'dcomeff']
# apply function to each column
df[cols_dt] = df[cols_dt].apply(convert_date_str_ser_to_datetime)

# Transform date-like columns and correct some dates

In [8]:
# convert some date columns to the next trading days (incl. the day itself)
cols_trading_day = ['dao', 'da', 'de', 'dw']
df[cols_trading_day] = df[cols_trading_day].apply(lambda x: get_trading_day_offset(x, 0))

In [9]:
# correct some dates
## dao
df.dao = np.minimum(df.dao, df.da)
df.dao = np.maximum(df.dao, get_trading_day_offset(df.da, -126))
## prior trading days
df.one_day = get_trading_day_offset(df.dao, -1)
df.aone_day = get_trading_day_offset(df.da, -1)
## create date of resolution
insert_cols(df, 'definitive_agt', 'dr', np.nan)
df.dr[df.statc.eq('C')] = df.de[df.statc.eq('C')]
df.dr[df.statc.eq('W')] = df.dw[df.statc.eq('W')]

# Correct some rows manually
These are database error / inconsistency that I found during analysis, mainly about the string format of deal consideration. Let us try to correct as many as possible though this can never be comprehensive.

In [10]:
# drop wrong data entries
df = df.drop(index = get_delete_index(df))

# correct consideration
cols = ['consid', 'consido']
df[cols] = correct_consid(df[cols])

# Save results

In [11]:
# hdf
filepath = f"{path}/data/intermediate/df_basic_cleaning.h5"

df.to_hdf(filepath, key = 'df', mode='w')

In [12]:
# csv
filepath2 = f"{path}/data/intermediate/df_basic_cleaning.csv"

df.to_csv(filepath2)